# Deploy your production ready TensorFlow v2.x models with classify signature.

## Training and Signtaure creation

In [1]:
import os

# TensorFlow and tf.keras
import tensorflow as tf
print(tf.__version__)

2.1.0


In [2]:
# This is for using the GPU efficiently
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import tensorflow as tf
from tensorflow.python.ops import lookup_ops
from tensorflow.python.ops import array_ops
from tensorflow.python.framework import dtypes
from tensorflow.python.saved_model.signature_def_utils_impl import \
    build_signature_def
from tensorflow.python.saved_model.signature_def_utils_impl import \
    is_valid_signature

from tensorflow.compat.v1.saved_model.utils import build_tensor_info

In [4]:
from tensorflow.python.saved_model import tag_constants, signature_constants
from tensorflow.python.saved_model import builder as saved_builder

from tensorflow.compat.v1.keras.backend import get_session

In [5]:
tf.compat.v1.disable_eager_execution()

In [6]:
# Get the dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# scale the values to 0.0 to 1.0
train_images = train_images / 255.0
test_images = test_images / 255.0

# reshape for feeding into the model
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

class_names = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print('\ntrain_images.shape: {}, of {}'.format(train_images.shape, train_images.dtype))
print('test_images.shape: {}, of {}'.format(test_images.shape, test_images.dtype))


train_images.shape: (60000, 28, 28, 1), of float64
test_images.shape: (10000, 28, 28, 1), of float64


In [7]:
model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(input_shape=(28,28,1), filters=8, kernel_size=3, 
                      strides=2, activation='relu', name='Conv1'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax, name='Softmax')
])
model.summary()

W0413 19:24:51.742632 139790839142208 deprecation.py:506] From /home/krxat/anaconda3/envs/tflow2.1/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 13, 13, 8)         80        
_________________________________________________________________
flatten (Flatten)            (None, 1352)              0         
_________________________________________________________________
Softmax (Dense)              (None, 10)                13530     
Total params: 13,610
Trainable params: 13,610
Non-trainable params: 0
_________________________________________________________________


In [8]:
def preprocess_image(image_buffer):
    image = tf.image.decode_jpeg(image_buffer, channels=1)
    image = tf.image.resize(image, (28, 28))
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)

    return image / 255

In [9]:
# Create the classify signature and the metagraph
NUM_CLASSES = len(class_names)

serialized_tf_example = array_ops.placeholder(tf.string,
                                              name='tf_example')
feature_configs = {'x': tf.io.FixedLenFeature([], tf.string), }
tf_example = tf.io.parse_example(serialized_tf_example,
                                 feature_configs)

jpegs = tf_example['x']
x = tf.map_fn(preprocess_image, jpegs, dtype=tf.float32)
y = model(x)

# Create Signature
values, indices = tf.nn.top_k(y, NUM_CLASSES)
table = lookup_ops.index_to_string_table_from_tensor(
    vocabulary_list=tf.constant(class_names),
    default_value="UNK",
    name=None
    )
prediction_classes = table.lookup(tf.cast(indices, dtype=dtypes.int64))

classification_inputs = build_tensor_info(serialized_tf_example)
classification_outputs_classes = build_tensor_info(prediction_classes)
classification_outputs_scores = build_tensor_info(values)

classification_signature = build_signature_def(
  inputs={
      tf.compat.v1.saved_model.signature_constants.CLASSIFY_INPUTS:
          classification_inputs
  },
  outputs={
      tf.compat.v1.saved_model.signature_constants
      .CLASSIFY_OUTPUT_CLASSES:
          classification_outputs_classes,
      tf.compat.v1.saved_model.signature_constants
      .CLASSIFY_OUTPUT_SCORES:
          classification_outputs_scores
  },
  method_name=tf.compat.v1.saved_model.signature_constants
  .CLASSIFY_METHOD_NAME)

# validating signtaure
valid_signature = is_valid_signature(classification_signature)
if not valid_signature:
    self.logger.error('Signature invalid')
    raise ValueError("Error: Classification signature not valid!")

W0413 19:24:52.154987 139790839142208 deprecation.py:323] From <ipython-input-9-c4a14d945962>:23: build_tensor_info (from tensorflow.python.saved_model.utils_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.


In [10]:
# This just to get a trained model. You need to create a better 
# model with more layers and optimize the options in real scenario
# to get good performance
epochs = 1

model.compile(optimizer=tf.optimizers.Adam(), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(train_images, train_labels, epochs=epochs)

Train on 60000 samples
60000/60000 [==============================] - 5s 91us/sample - loss: 0.5487 - accuracy: 0.8096


In [11]:
# Delete if there is already a model existing at the path
export_path = './models/123'
if os.path.isdir(export_path):
    print('\nAlready saved a model, cleaning up\n')
    !rm -r {export_path}

session = get_session()
builder = saved_builder.SavedModelBuilder(export_path)

# Add the meta_graph and the variables to the builder
serv_key = signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY
builder.add_meta_graph_and_variables(
      session, [tag_constants.SERVING],
      signature_def_map={
           serv_key: classification_signature,
      },
      main_op=tf.compat.v1.tables_initializer(),
      strip_default_attrs=True)
# save the graph
builder.save()


Already saved a model, cleaning up



b'./models/123/saved_model.pb'

- - - -

## Deploy the model

### Run the docker server
Refer to the blog for more details

### Send request and get the outputs

In [12]:
import requests
import base64
import json

In [13]:
headers = {"content-type": "application/json"}
SERVER_URL = 'http://localhost:8501/v1/models/fashion:classify'

loaded_image = open('Dinning_val_3.jpg', 'rb').read()
jpeg_bytes = base64.b64encode(loaded_image).decode('utf-8')

In [14]:
body = {
    "signature_name": "serving_default",
    "examples" : [{
        "x": { "b64": jpeg_bytes},  
     }]
}

In [15]:
r = requests.post(SERVER_URL, data=json.dumps(body), headers = headers)

In [16]:
print(r)

<Response [200]>


In [17]:
json.loads(r.text)

{'results': [[['Bag', 0.999858618],
   ['Shirt', 6.90536544e-05],
   ['T-shirt/top', 4.54395085e-05],
   ['Trouser', 2.42908627e-05],
   ['Dress', 2.16052649e-06],
   ['Ankle boot', 3.70926585e-07],
   ['Pullover', 3.7451251e-08],
   ['Coat', 1.59411251e-09],
   ['Sneaker', 9.65969305e-10],
   ['Sandal', 1.49970661e-11]]]}